In [ ]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio

import base64
from PIL import Image
from io import BytesIO

import numpy as np

import matplotlib.pyplot as plt
from keras.models import load_model

In [ ]:
model = load_model("mymodel.h5")

In [ ]:
sio = socketio.Server()
app = Flask(__name__)

In [ ]:
def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)

In [ ]:
def process_image(img):
    #160,320,3 => 60,80,3
    return img[10:130:2, ::4, :]

In [ ]:
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]
        throttle_prev = data["throttle"]

        decoded = base64.b64decode(image_str)
        image = Image.open(BytesIO(decoded))
        
        img_array = np.asarray(image)
        img = process_image(img_array)
        img_batch = np.expand_dims(img, axis=0)
        
        steering_angle = float(model.predict(img_batch))
        
        if speed < 25.0:
            throttle = 0.5
        else:
            throttle = -0.1

        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)
        
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)